# Agent

사용자가 Agent에게 어떤 작업을 시켰을때 개발자가 만들어낸 Tool의 기능을 가지고 스스로 생각하여 필요한 Tool들을 가지고 실행하여 결과물을 냄

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [1]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH15-Agents-practice")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Agents-practice


### 1. 도구를 생성한다.

In [3]:
from langchain.tools import tool
from typing import List, Dict, Annotated
from langchain_teddynote.tools import GoogleNews
from langchain_experimental.utilities import PythonREPL


# 도구 생성
@tool
def search_news(query: str) -> List[Dict[str, str]]:
    """Search Google News by input keyword"""
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)


# 도구 생성
@tool
def python_repl_tool(
    code: Annotated[str, "The python code to execute to generate your chart."],
):
    """Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user."""
    result = ""
    try:
        result = PythonREPL().run(code)
    except BaseException as e:
        print(f"Failed to execute. Error: {repr(e)}")
    finally:
        return result


print(f"도구 이름: {search_news.name}")
print(f"도구 설명: {search_news.description}")
print(f"도구 이름: {python_repl_tool.name}")
print(f"도구 설명: {python_repl_tool.description}")

도구 이름: search_news
도구 설명: Search Google News by input keyword
도구 이름: python_repl_tool
도구 설명: Use this to execute python code. If you want to see the output of a value,
    you should print it out with `print(...)`. This is visible to the user.


### 1.1 생성된 도구를 리스트에 담는다.

In [4]:
# tools 정의
tools = [search_news, python_repl_tool]

### 2. Agent 프롬프트를 생성한다.

프롬프트의 내용으로는 Agent가 Tool을 사용할 수 있도록 유도하는 내용으로 한다.

(1) "system"

시스템 지침 (역할 규정) → 모델에게 “당신은 뉴스 검색을 도와주는 어시스턴트이고, 뉴스를 찾을 때는 반드시 search_news 툴을 사용하라”라고 규칙을 주는 부분입니다.

(2) "placeholder", "{chat_history}"

이전 대화 기록이 들어가는 자리.

에이전트가 과거 대화를 기억하고 연속적으로 대화할 수 있도록 합니다.

실행 시점에 chat_history 값으로 실제 대화 로그가 채워집니다.

(3) "human", "{input}"

사용자가 실제로 입력하는 메시지.

코드 실행할 때 {"input": "닌텐도 관련 최신 뉴스 알려줘"} 같은 값이 들어갑니다.

(4) "placeholder", "{agent_scratchpad}"

에이전트의 중간 사고 과정/도구 호출 기록이 들어가는 자리.

예: 모델이 “이 질문은 뉴스 검색이 필요하다 → search_news 툴 실행 → 결과 확인” 같은 중간 reasoning 및 호출 기록을 남길 수 있음.

Agent가 체인처럼 여러 단계를 거칠 때 반드시 필요한 부분이에요.

In [6]:
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 생성
# 프롬프트는 에이전트에게 모델이 수행할 작업을 설명하는 텍스트를 제공합니다. (도구의 이름과 역할을 입력)
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `search_news` tool for searching keyword related news.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        # 에이전트의 중간 사고과정/도구호출 기록이 들어갈 자리
        ("placeholder", "{agent_scratchpad}"),
    ]
)

### 3. Agent를 생성한다.

의미

에이전트를 만드는 방법은 여러 가지가 있는데,

create_openai_functions_agent

create_react_agent

create_tool_calling_agent
… 등이 있습니다.

그중 create_tool_calling_agent 는 OpenAI의 함수 호출(function calling) 또는 tool calling API를 활용해서, LLM이 스스로 어떤 툴을 쓸지 선택하고 호출할 수 있게 하는 에이전트를 만들어 줍니다.

In [7]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

### 4. Agent Executor를 생성하고 Agent모델을 연결시킨다.

In [ ]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(
    # agent 모델 연결
    agent=agent,
    
    # Tool들 연결
    tools=tools,
    
    # 실행 과정을 자세히 로그로 출력할지 여부
    verbose=True,
    
    # Agent가 툴을 몇 번까지 연속 호출할 수 있는지 제한.
    # 혹시 무한루프 빠지면 10번째에 종료.
    max_iterations=10,
    
    # 실행 제한 시간(초 단위). Agent가 10초 넘게 실행되면 강제 중단.
    max_execution_time=10,
    
    # Agent가 도구 호출 결과를 파싱하다가 에러가 나도 프로그램이 죽지 않고 자동으로 복구(retry) 하도록 함.
    handle_parsing_errors=True,
)

In [10]:
# AgentExecutor 실행
result = agent_executor.invoke({"input": "닌텐도 스위치에 대한 뉴스를 검색해 주세요. 그 다음 나온 뉴스들의 제목을 오름차순으로 출력해주세요"})

print("Agent 실행 결과:")
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `search_news` with `{'query': '닌텐도 스위치'}`


[{'url': 'https://news.google.com/rss/articles/CBMia0FVX3lxTE9SejdGX0dRVWRTY1ptVFhqa1g0aWJXUlNiT1hUV3NLalNBTlFjV0xOYzJKNUFhYTM2S1VCYUM3M2F4SUh5RVdoM2VGOXctcU1semtyVldXZDBmQjhpekJjeXo4XzY1M0tzRGxr?oc=5', 'content': "'닌텐도 스위치 2' 쾌적한 휴대 위한 액세서리 모음 - 디지털포스트(PC사랑)"}, {'url': 'https://news.google.com/rss/articles/CBMic0FVX3lxTE01c0otcl9XVlZrcGNaaWJGZXZickYyVWFXcnlsTVpkbHplcWl2Tm8wRnA1bmY4bnU3M2c3T0J2OHdXenY3YWc1a3pMcmhMSkNoUWNKeEZDTkxsVzJXNVhmTkZCeWFsYnYtaF9jdnNnV3FMR1k?oc=5', 'content': '닌텐도, 불법 개조 스위치 판매자에 승소…200만달러 배상 명령 - 디지털투데이'}, {'url': 'https://news.google.com/rss/articles/CBMiYEFVX3lxTFAzUWZkTGtELV8zWWdEa3lfcGRuTk1iWDJ1NDIxT1ZiTVlHREhVS2tEdnVmWTN5dXZkdXhXdk1hM252RVlGNHVMMURPeWNGbGFZYzJDTTV0OHZEczBsdEVxMg?oc=5', 'content': '닌텐도 불법 복제 업자에 승소…28억 원 지불 명령 내려져 - 네이트'}, {'url': 'https://news.google.com/rss/articles/CBMiQkFVX3lxTE9zWWpVaURqQTI2OTBCX1hoLUdWXy02cV9RNUU4elQtT2RoMVBqd053dmFGXzR2N0tvb

Python REPL can execute arbitrary code. Use with caution.



Invoking: `python_repl_tool` with `{'code': 'news_titles = [\n    "닌텐도 스위치 2\' 쾌적한 휴대 위한 액세서리 모음 - 디지털포스트(PC사랑)",\n    "닌텐도, 불법 개조 스위치 판매자에 승소…200만달러 배상 명령 - 디지털투데이",\n    "닌텐도 불법 복제 업자에 승소…28억 원 지불 명령 내려져 - 네이트",\n    "닌텐도 스위치2 리뷰, 결국은 사게 됩니다 - 디에디트",\n    "「Nintendo Switch 인디 게임 세일」이 8/19~9/1 기간 동안 진행됩니다. | News | 한국닌텐도주식회사 - Nintendo"\n]\n\n# Sort the titles in ascending order\nsorted_titles = sorted(news_titles)\n\nsorted_titles'}`


다음은 닌텐도 스위치에 대한 뉴스 제목을 오름차순으로 정렬한 목록입니다:

1. 「Nintendo Switch 인디 게임 세일」이 8/19~9/1 기간 동안 진행됩니다. | News | 한국닌텐도주식회사 - Nintendo
2. 닌텐도, 불법 개조 스위치 판매자에 승소…200만달러 배상 명령 - 디지털투데이
3. 닌텐도 불법 복제 업자에 승소…28억 원 지불 명령 내려져 - 네이트
4. 닌텐도 스위치 2' 쾌적한 휴대 위한 액세서리 모음 - 디지털포스트(PC사랑)
5. 닌텐도 스위치2 리뷰, 결국은 사게 됩니다 - 디에디트

> Finished chain.
Agent 실행 결과:
다음은 닌텐도 스위치에 대한 뉴스 제목을 오름차순으로 정렬한 목록입니다:

1. 「Nintendo Switch 인디 게임 세일」이 8/19~9/1 기간 동안 진행됩니다. | News | 한국닌텐도주식회사 - Nintendo
2. 닌텐도, 불법 개조 스위치 판매자에 승소…200만달러 배상 명령 - 디지털투데이
3. 닌텐도 불법 복제 업자에 승소…28억 원 지불 명령 내려져 - 네이트
4. 닌텐도 스위치 2' 

### Stream 출력으로 단계별 결과 확인

Stream을 사용하면 중간의 내용이 출력되며 이를 시각화하면 디버깅하기에 수월하다.

In [11]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
)

In [13]:
# 스트리밍 모드로 실행합니다
result = agent_executor.stream({"input": "닌텐도 스위치에 대한 뉴스를 검색해 주세요. 그 다음 나온 뉴스들의 제목을 오름차순으로 출력해주세요"})

for step in result:
    # 중간 단계 출력
    print(step)
    print("===" * 20)

{'actions': [ToolAgentAction(tool='search_news', tool_input={'query': '닌텐도 스위치'}, log="\nInvoking: `search_news` with `{'query': '닌텐도 스위치'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Kd5EucQ5XPj1EwWFCwr7uvUv', 'function': {'arguments': '{"query":"닌텐도 스위치"}', 'name': 'search_news'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'service_tier': 'default'}, id='run--860c5d58-a6ad-4b53-884a-a0d75c92685b', tool_calls=[{'name': 'search_news', 'args': {'query': '닌텐도 스위치'}, 'id': 'call_Kd5EucQ5XPj1EwWFCwr7uvUv', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'search_news', 'args': '{"query":"닌텐도 스위치"}', 'id': 'call_Kd5EucQ5XPj1EwWFCwr7uvUv', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_Kd5EucQ5XPj1EwWFCwr7uvUv')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 

In [19]:
# AgentCallbacks와 AgentStreamParser를 langchain_teddynote.messages에서 가져옵니다.
from langchain_teddynote.messages import AgentCallbacks, AgentStreamParser


# 도구 호출 시 실행되는 콜백 함수입니다.
def tool_callback(tool) -> None:
    print("<<<<<<< 도구 호출 >>>>>>")
    print(f"Tool: {tool.get('tool')}")  # 사용된 도구의 이름을 출력합니다.
    print("<<<<<<< 도구 호출 >>>>>>")


# 관찰 결과를 출력하는 콜백 함수입니다.
def observation_callback(observation) -> None:
    print("<<<<<<< 관찰 내용 >>>>>>")
    print(
        f"Observation: {observation.get('observation')[0]}"
    )  # 관찰 내용을 출력합니다.
    print("<<<<<<< 관찰 내용 >>>>>>")


# 최종 결과를 출력하는 콜백 함수입니다.
def result_callback(result: str) -> None:
    print("<<<<<<< 최종 답변 >>>>>>")
    print(result)  # 최종 답변을 출력합니다.
    print("<<<<<<< 최종 답변 >>>>>>")


# AgentCallbacks 객체를 생성하여 각 단계별 콜백 함수를 설정합니다.
agent_callbacks = AgentCallbacks(
    tool_callback=tool_callback,
    observation_callback=observation_callback,
    result_callback=result_callback,
)

# AgentStreamParser 객체를 생성하여 에이전트의 실행 과정을 파싱합니다.
agent_stream_parser = AgentStreamParser(agent_callbacks)

In [21]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream({"input": "k-pop 데몬헌터스에 대한 뉴스를 검색해 주세요."})

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

<<<<<<< 도구 호출 >>>>>>
Tool: search_news
<<<<<<< 도구 호출 >>>>>>
<<<<<<< 관찰 내용 >>>>>>
Observation: {'url': 'https://news.google.com/rss/articles/CBMiYEFVX3lxTFBQbFlRZU5oMnZQQllRdGRUcGQzTy16REVjUWVXaW5YM1ZCTHlkLXdKWnJKVUpMazJnZVVNSlRKWVY4eE5ud3hxLXVWdzRnbUVRLXZwWnduSUFFUV9qN3RWSdIBeEFVX3lxTE1DcUQyRzIwQ2QwTkpBZ3o2RnN4QWNKSlRlMHR4R3NTSUdHSkRmM3BnSFFRcG5KUHBZSWp5MkxOM0lIel9POUplZWdsZlRPZ1I1MHNYNTNCOGxGaTZnWkk0RWswV1Bwal9TSE91VWoyeW95TDVEd3c0Zw?oc=5', 'content': "케데헌 '골든', 빌보드 '핫100' 통산 4주 1위 - 뉴시스"}
<<<<<<< 관찰 내용 >>>>>>
<<<<<<< 최종 답변 >>>>>>
다음은 K-pop 데몬헌터스에 대한 최근 뉴스입니다:

1. [케데헌 '골든', 빌보드 '핫100' 통산 4주 1위 - 뉴시스](https://news.google.com/rss/articles/CBMiYEFVX3lxTFBQbFlRZU5oMnZQQllRdGRUcGQzTy16REVjUWVXaW5YM1ZCTHlkLXdKWnJKVUpMazJnZVVNSlRKWVY4eE5ud3hxLXVWdzRnbUVRLXZwWnduSUFFUV9qN3RWSdIBeEFVX3lxTE1DcUQyRzIwQ2QwTkpBZ3o2RnN4QWNKSlRlMHR4R3NTSUdHSkRmM3BnSFFRcG5KUHBZSWp5MkxOM0lIel9POUplZWdsZlRPZ1I1MHNYNTNCOGxGaTZnWkk0RWswV1Bwal9TSE91VWoyeW95TDVEd3c0Zw?oc=5)

2. [‘케이팝 데몬 헌터스’, 영국 소비까지 잡았다 - 이투데이](https://n

### 너무 많은 호출로 IndexError: string index out of range

In [23]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream({"input": "k-pop 데몬헌터스에 대한 뉴스를 검색해 주세요. 그 후 나온 뉴스들의 제목을 내림차순으로 정렬해주세요"})

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)

<<<<<<< 도구 호출 >>>>>>
Tool: search_news
<<<<<<< 도구 호출 >>>>>>
<<<<<<< 관찰 내용 >>>>>>
Observation: {'url': 'https://news.google.com/rss/articles/CBMiYEFVX3lxTFBQbFlRZU5oMnZQQllRdGRUcGQzTy16REVjUWVXaW5YM1ZCTHlkLXdKWnJKVUpMazJnZVVNSlRKWVY4eE5ud3hxLXVWdzRnbUVRLXZwWnduSUFFUV9qN3RWSdIBeEFVX3lxTE1DcUQyRzIwQ2QwTkpBZ3o2RnN4QWNKSlRlMHR4R3NTSUdHSkRmM3BnSFFRcG5KUHBZSWp5MkxOM0lIel9POUplZWdsZlRPZ1I1MHNYNTNCOGxGaTZnWkk0RWswV1Bwal9TSE91VWoyeW95TDVEd3c0Zw?oc=5', 'content': "케데헌 '골든', 빌보드 '핫100' 통산 4주 1위 - 뉴시스"}
<<<<<<< 관찰 내용 >>>>>>
<<<<<<< 도구 호출 >>>>>>
Tool: python_repl_tool
<<<<<<< 도구 호출 >>>>>>
<<<<<<< 관찰 내용 >>>>>>
Observation: S
<<<<<<< 관찰 내용 >>>>>>
<<<<<<< 도구 호출 >>>>>>
Tool: python_repl_tool
<<<<<<< 도구 호출 >>>>>>
<<<<<<< 관찰 내용 >>>>>>
Observation: S
<<<<<<< 관찰 내용 >>>>>>
<<<<<<< 도구 호출 >>>>>>
Tool: python_repl_tool
<<<<<<< 도구 호출 >>>>>>
<<<<<<< 관찰 내용 >>>>>>


IndexError: string index out of range

### 대화 기록형 Agent

In [126]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
@tool
def add(a: int, b: int) -> int:
    """두 수를 더하는 도구"""
    return a + b

@tool
def subtract(a: int, b: int) -> int:
    """두 수를 빼는 도구"""
    return a - b

@tool
def multiply(a: int, b: int) -> int:
    """두 수를 곱하는 도구"""
    return a * b

@tool
def divide(a: int, b: int) -> float:
    """두 수를 나누는 도구 (0으로 나누기 불가)"""
    if b == 0:
        return float("inf")  # 또는 에러 메시지 반환
    return a / b

tools = [add, multiply, subtract, divide]

In [148]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 수학을 푸는 에이전트야. 필요할 때 add, multiply 도구를 사용해."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [150]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

In [151]:
# AgentCallbacks와 AgentStreamParser를 langchain_teddynote.messages에서 가져옵니다.
from langchain_teddynote.messages import AgentCallbacks, AgentStreamParser


# 도구 호출 시 실행되는 콜백 함수입니다.
def tool_callback(tool) -> None:
    print("<<<<<<< 도구 호출 >>>>>>")
    print(f"Tool: {tool.get('tool')}")  # 사용된 도구의 이름을 출력합니다.
    print("<<<<<<< 도구 호출 >>>>>>")


# 최종 결과를 출력하는 콜백 함수입니다.
def result_callback(result: str) -> None:
    print("<<<<<<< 최종 답변 >>>>>>")
    print(result)  # 최종 답변을 출력합니다.
    print("<<<<<<< 최종 답변 >>>>>>")


# AgentCallbacks 객체를 생성하여 각 단계별 콜백 함수를 설정합니다.
agent_callbacks = AgentCallbacks(
    tool_callback=tool_callback,
    result_callback=result_callback,
)

# AgentStreamParser 객체를 생성하여 에이전트의 실행 과정을 파싱합니다.
agent_stream_parser = AgentStreamParser(agent_callbacks)

In [152]:
config = {"configurable": {"session_id": "abc123"}}

In [153]:
# 첫 번째 질문
result1 = agent_with_chat_history.stream({"input": "3+7이 뭐야?"}, config=config)
for step in result1:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
<<<<<<< 도구 호출 >>>>>>
Tool: add
<<<<<<< 도구 호출 >>>>>>

Invoking: `add` with `{'a': 3, 'b': 7}`


10[관찰 내용]
Observation: 10
3 + 7은 10입니다.

> Finished chain.
<<<<<<< 최종 답변 >>>>>>
3 + 7은 10입니다.
<<<<<<< 최종 답변 >>>>>>


In [154]:
# 두 번째 질문 (앞의 결과 활용)
result2 = agent_with_chat_history.stream({"input": "그다음 내가 나온 값에 10을 곱하면 뭐야?"}, config=config)
for step in result2:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
<<<<<<< 도구 호출 >>>>>>
Tool: multiply
<<<<<<< 도구 호출 >>>>>>

Invoking: `multiply` with `{'a': 10, 'b': 10}`


100[관찰 내용]
Observation: 100
10에 10을 곱하면 100입니다.

> Finished chain.
<<<<<<< 최종 답변 >>>>>>
10에 10을 곱하면 100입니다.
<<<<<<< 최종 답변 >>>>>>


### 

### 3번째 수식에서 6/6을 해야하지만 나눗셈을 하지않고 b값도 틀린 값을 넣어서 계산 결과가 엉망이다.

In [135]:
# 세 번째 질문 (앞의 결과 활용)
result3 = agent_with_chat_history.stream({"input": "6÷2(1+2) 는 뭘까?"}, config=config)
for step in result3:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
<<<<<<< 도구 호출 >>>>>>
Tool: add
<<<<<<< 도구 호출 >>>>>>

Invoking: `add` with `{'a': 1, 'b': 2}`


3[관찰 내용]
Observation: 3
<<<<<<< 도구 호출 >>>>>>
Tool: multiply
<<<<<<< 도구 호출 >>>>>>

Invoking: `multiply` with `{'a': 2, 'b': 3}`


6[관찰 내용]
Observation: 6
<<<<<<< 도구 호출 >>>>>>
Tool: multiply
<<<<<<< 도구 호출 >>>>>>

Invoking: `multiply` with `{'a': 6, 'b': 1}`


6[관찰 내용]
Observation: 6
6 ÷ 2(1 + 2)의 계산 결과는 6입니다.

> Finished chain.
<<<<<<< 최종 답변 >>>>>>
6 ÷ 2(1 + 2)의 계산 결과는 6입니다.
<<<<<<< 최종 답변 >>>>>>


In [155]:
# 네 번째 질문 (앞의 결과 활용)
result4 = agent_with_chat_history.stream({"input": "방금나온값과 그 전에 나온값을 더하면 뭘까?"}, config=config)
for step in result4:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
<<<<<<< 도구 호출 >>>>>>
Tool: add
<<<<<<< 도구 호출 >>>>>>

Invoking: `add` with `{'a': 10, 'b': 100}`


110[관찰 내용]
Observation: 110
방금 나온 값과 그 전에 나온 값을 더하면 110입니다.

> Finished chain.
<<<<<<< 최종 답변 >>>>>>
방금 나온 값과 그 전에 나온 값을 더하면 110입니다.
<<<<<<< 최종 답변 >>>>>>


### 나눗셈을 하지않고 곱하기로 하려는 모습. 에러가 난다.

이유를 찾음 -> 프롬프트에 필요할때 add, multiply 도구를 사용하라 했기때문에 divide tool은 사용하지 않았음
진짜 이유를 찾음 -> tools = [add, multiply, subtract, divide] 여기서 subtract, divide를 넣지 않고 사용했었따. 바보다.

In [158]:
# 마지막 질문 (앞의 결과 활용)
result5 = agent_with_chat_history.stream({"input": "방금나온값을 2로 나누면?"}, config=config)
for step in result5:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
<<<<<<< 도구 호출 >>>>>>
Tool: divide
<<<<<<< 도구 호출 >>>>>>

Invoking: `divide` with `{'a': 27, 'b': 2}`


13.5[관찰 내용]
Observation: 13.5
방금 나온 값을 2로 나누면 13.5입니다.

> Finished chain.
<<<<<<< 최종 답변 >>>>>>
방금 나온 값을 2로 나누면 13.5입니다.
<<<<<<< 최종 답변 >>>>>>


In [159]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 수학을 푸는 에이전트야. 필요할 때 add, subtract, multiply, divide 도구를 사용해. input값과 output값은 모두 정수야"),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

### ~~나눗셈을 자꾸 multiply로 사용하려고 함.~~

In [161]:
config = {"configurable": {"session_id": "abc123"}}

In [162]:
result1 = agent_with_chat_history.stream({"input": "10나누기5는 뭐야?"}, config=config)
for step in result1:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
<<<<<<< 도구 호출 >>>>>>
Tool: divide
<<<<<<< 도구 호출 >>>>>>

Invoking: `divide` with `{'a': 10, 'b': 5}`


2.0[관찰 내용]
Observation: 2.0
10 나누기 5는 2입니다.

> Finished chain.
<<<<<<< 최종 답변 >>>>>>
10 나누기 5는 2입니다.
<<<<<<< 최종 답변 >>>>>>


In [163]:
result2 = agent_with_chat_history.stream({"input": "방금나온값을 2로 나눈값은 뭐야?"}, config=config)
for step in result2:
    agent_stream_parser.process_agent_steps(step)



> Entering new None chain...
<<<<<<< 도구 호출 >>>>>>
Tool: divide
<<<<<<< 도구 호출 >>>>>>

Invoking: `divide` with `{'a': 2, 'b': 2}`


1.0[관찰 내용]
Observation: 1.0
방금 나온 값을 2로 나누면 1입니다.

> Finished chain.
<<<<<<< 최종 답변 >>>>>>
방금 나온 값을 2로 나누면 1입니다.
<<<<<<< 최종 답변 >>>>>>
